## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Data/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Calabazar De Sagua,CU,22.6447,-79.8961,82.87,83,40,8.05,scattered clouds
1,1,Antofagasta,CL,-23.6500,-70.4000,57.90,82,100,6.04,overcast clouds
2,2,Yertarskiy,RU,56.7902,64.2995,34.23,95,82,5.66,broken clouds
3,3,Arraial Do Cabo,BR,-22.9661,-42.0278,71.37,88,15,27.67,few clouds
4,4,Galyugayevskaya,RU,43.6969,44.9344,58.14,62,47,5.35,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Antofagasta,CL,-23.6500,-70.4000,57.90,82,100,6.04,overcast clouds
3,3,Arraial Do Cabo,BR,-22.9661,-42.0278,71.37,88,15,27.67,few clouds
4,4,Galyugayevskaya,RU,43.6969,44.9344,58.14,62,47,5.35,scattered clouds
6,6,Karratha,AU,-20.7377,116.8463,76.06,25,100,25.72,overcast clouds
8,8,Avarua,CK,-21.2078,-159.7750,78.85,69,75,6.91,broken clouds
9,9,Jamestown,US,42.0970,-79.2353,72.72,87,98,9.57,overcast clouds
11,11,Chacabuco,AR,-34.6417,-60.4739,56.03,74,37,9.60,scattered clouds
13,13,Cabedelo,BR,-6.9811,-34.8339,76.89,72,37,14.41,scattered clouds
16,16,Kununurra,AU,-15.7667,128.7333,78.78,69,46,5.75,scattered clouds
17,17,Rikitea,PF,-23.1203,-134.9692,74.01,77,6,13.53,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Antofagasta,CL,57.90,overcast clouds,-23.6500,-70.4000,
3,Arraial Do Cabo,BR,71.37,few clouds,-22.9661,-42.0278,
4,Galyugayevskaya,RU,58.14,scattered clouds,43.6969,44.9344,
6,Karratha,AU,76.06,overcast clouds,-20.7377,116.8463,
8,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,
9,Jamestown,US,72.72,overcast clouds,42.0970,-79.2353,
11,Chacabuco,AR,56.03,scattered clouds,-34.6417,-60.4739,
13,Cabedelo,BR,76.89,scattered clouds,-6.9811,-34.8339,
16,Kununurra,AU,78.78,scattered clouds,-15.7667,128.7333,
17,Rikitea,PF,74.01,clear sky,-23.1203,-134.9692,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_df = hotel_df.dropna()
clean_df.count()

City                   280
Country                280
Max Temp               280
Current Description    280
Lat                    280
Lng                    280
Hotel Name             280
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
output_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_df.to_csv(output_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))